In [117]:
import pandas as pd
encoding = 'iso-8859-8'
# encoding =None
def file_election(election):
    name = 'results_' +str(election)

    if election< 21:
        df = pd.read_excel(name + '.xls')

    if election>=21 and election<25:
        df = pd.read_csv(name+'.csv',encoding='iso-8859-8')

    if election==25:
        df = pd.read_csv(name+'.csv')
    df.columns = [col.strip(' ') for col in df.columns]
    df = df.rename({'מספר קלפי':'קלפי' ,'סמל קלפי':'קלפי' ,'ב ז ב':'בזב'},axis='columns' )


    return df

files = []
timeline = pd.read_excel('timeline.xlsx')
for elc in range(14,26):
    df = file_election(elc)
    col = list(df.columns)
    df['election'] = elc
    files.append(df[['election']+col])

df = pd.concat(files)
df = df.merge(timeline,on='election', how='left')
df = df.drop(["ברזל","ריכוז",'שופט','סמל ועדה','נפה','פיצול','כתובת'],axis=1)

df_groupby = df.groupby('election')[df.columns[4:-3]].sum().div(df.groupby('election')[df.columns[5]].sum(),axis=0).round(3)*100
df = df.drop(df_groupby.sum()[(df_groupby.max()<0.9)].index,axis=1)

df.head()

,election,סמל ישוב,קלפי,שם ישוב,בזב,מצביעים,כשרים,פסולים,אמת,ב,...,כ,נר,צפ,ודעם,דעם,ום,כף,year,month,day
0,14,7,1.0,שחר,314.0,256.0,256.0,0.0,103.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999,5,17
1,14,10,1.0,תירוש,260.0,203.0,198.0,5.0,5.0,74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999,5,17
2,14,11,1.0,"ניר ח""ן",229.0,148.0,146.0,2.0,47.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999,5,17
3,14,13,1.0,חצבה,450.0,318.0,316.0,2.0,172.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999,5,17
4,14,15,1.0,נועם,288.0,232.0,230.0,2.0,6.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999,5,17
